In [116]:
from typing import Tuple

#from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
torch.manual_seed(99)

from matplotlib import pyplot as plot

In [117]:
df = pd.read_csv("./data/datasets/historical.data")
#df = df.set_index("date").sort_index(ascending=True)
df = df.sort_values(by="date",ascending=True)
df["month"] = df.apply(lambda x: int(x["date"].split("-")[1])-1, axis=1) # category indexed to 0
df["day"] = df.apply(lambda x: int(x["date"].split(" ")[0].split("-")[2])-1, axis=1) # category indexed to 0
df["hour"] = df.apply(lambda x: int(x["date"].split(" ")[1].split(":")[0]), axis=1)
df["minute"] = df.apply(lambda x: int(x["date"].split(" ")[1].split(":")[1]), axis=1)
df = df.drop(["date"], axis=1)
#df.apply(lambda x: x["date"].split(" ")[0].split("-")[1] + "-" + x["date"].split(" ")[0].split("-")[2], axis=1)
#display(df)
df_inside = df.loc[df["label"] == "inside"].drop(["label"], axis=1)
df_outside = df.loc[df["label"] == "outside"].drop(["label"], axis=1)
print("inside")
display(df_inside)
#print("outside")
#display(df_outside)

inside


,temperature,month,day,hour,minute
1,22.875,1,21,0,30
2,22.937,1,21,1,0
4,23.062,1,21,1,30
6,23.125,1,21,2,0
8,23.187,1,21,2,30
...,...,...,...,...,...
97001,24.812,10,29,21,30
97002,24.750,10,29,22,0
97005,24.500,10,29,22,30
97007,24.437,10,29,23,0


In [118]:



inside_arr=np.array(df_inside)
outside_arr=np.array(df_outside)

#print(inside_arr)

#train_test_split(inside_arr, shuffle=False)

def split_data(arr: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    total_len,_=arr.shape

    train_p = 0.80
    test_p = 0.10
    ivs_p = 0.10

    train_len = round(total_len*train_p)
    test_len = round(total_len*test_p)
    ivs_len = round(total_len*ivs_p)

    model_len = train_len+test_len

    total_used_len = model_len+ivs_len

    train = arr[:train_len]
    test = arr[train_len:model_len]
    ivs = arr[model_len:total_used_len]

    return train, test, ivs


inside_train, inside_test, inside_ivs = split_data(inside_arr)

print("train")
print(inside_train)
print("test")
print(inside_test)
print("ivs")
print(inside_ivs)

train
[[22.875  1.    21.     0.    30.   ]
 [22.937  1.    21.     1.     0.   ]
 [23.062  1.    21.     1.    30.   ]
 ...
 [23.375  4.    12.     8.     0.   ]
 [23.437  4.    12.     8.    30.   ]
 [23.375  4.    12.     9.     0.   ]]
test
[[23.375  4.    12.     9.    30.   ]
 [23.437  4.    12.    10.     0.   ]
 [23.375  4.    12.    10.    30.   ]
 ...
 [25.125  7.    21.     3.    30.   ]
 [25.062  7.    21.     4.     0.   ]
 [25.062  7.    21.     4.    30.   ]]
ivs
[[25.125  7.    21.     5.     0.   ]
 [25.062  7.    21.     5.    30.   ]
 [25.125  7.    21.     6.     0.   ]
 ...
 [24.5   10.    29.    22.    30.   ]
 [24.437 10.    29.    23.     0.   ]
 [24.562 10.    29.    23.    30.   ]]


In [119]:
# 2h30

per_min = (2*60) + 30
per_30_mins = per_min / 30

print("Number of Wanted Predictions: " + str(per_30_mins))

Number of Wanted Predictions: 5.0


One-step-ahead (auto-regressive)
<br>
One-step-ahead (auto-regressive with exogenous inputs)
<br>
Recursive
<br>
MIMO
<br>
Direct
<br>
Hybrid (Recursive and Direct)
<br>
Hybrid (Direct and MIMO)
<br>
<br>
MIMO is good too, but it would be more accurate and faster to compute several dedicated models for each step instead of all in a batch.
<br>
Direct and Hybrid(Direct and MIMO) are on top - they can execute all
<br>
<br>
Hybrid(Direct and MIMO) = [2,2,1] predictive models
<br>
Direct = [+-,1,1,1,1] predictive models
<br>
<br>
Other tests that could be implemented would be creating a Long Short-Term Memory model, trying out some kind of BERT model with months as categorial variables, or classical statistic models like SVMs or Decision Trees.

In [120]:
inside_train_0 = []
inside_train_X_0 = []
inside_train_Y_0 = []

imput_dim_df = 48 # 24H input

_temp_arr = []
_n = 0
for temp, month, day, hour, minute in inside_train:
    if len(_temp_arr) < imput_dim_df:
        _temp_arr.append(temp)
        _n+=1
        continue
    else:
        _temp_arr.pop(0)
        _temp_arr.append(temp)

    if _n < len(inside_train)-1:
        inside_train_X_0.append(_temp_arr)
        inside_train_Y_0.append(inside_train[_n+1][0])
        
        inside_train_0.append((torch.Tensor(_temp_arr), inside_train[_n+1][0]))

    _n+=1

inside_train_X_0 = torch.from_numpy(np.array(inside_train_X_0, dtype=np.float16))
inside_train_Y_0 = torch.from_numpy(np.array(inside_train_Y_0, dtype=np.float16))


In [121]:
print(inside_train_X_0.size())
print(inside_train_Y_0.size())

torch.Size([38661, 48])
torch.Size([38661])


In [122]:
batch_size = 1
n_iters = 38661
num_epochs = n_iters / (len(inside_train_X_0) / batch_size)
num_epochs = int(num_epochs)
print(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=inside_train_0, 
                                           batch_size=batch_size, 
                                           shuffle=True)

1


In [123]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your LSTM
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # 28 time steps
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out

In [124]:
input_dim = 48
hidden_dim = 48
layer_dim = 1
output_dim = 1

In [125]:
model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)

In [126]:
criterion = nn.CrossEntropyLoss()

In [127]:
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [128]:
len(list(model.parameters()))

6

In [129]:
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

torch.Size([192, 48])
torch.Size([192, 48])
torch.Size([192])
torch.Size([192])
torch.Size([1, 48])
torch.Size([1])


In [130]:
test_loader = [] # TODO 

In [131]:
# Number of steps to unroll
seq_dim = 1  

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as a torch tensor with gradient accumulation abilities
        images = images.view(-1, seq_dim, input_dim).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels.long())

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Resize images
                images = images.view(-1, seq_dim, input_dim)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

IndexError: Target 22 is out of bounds.